# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Vector{Float64}:
 0.48218786158320404
 0.9650715209782048
 0.39369361808888037
 0.7085975306850376
 0.28155863280552784
 0.7584745340377707
 0.41112486980573804
 0.6468988084740505
 0.3280938515373343
 0.10657434269088573
 ⋮
 0.9838021508800607
 0.3929225437608479
 0.4753878538563975
 0.5564961049475811
 0.15292852460476758
 0.1450381915483867
 0.0635522544890813
 0.032713259659018834
 0.7022211354155339

In [2]:
sum(a)

5.001462617496209e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [3]:
@time sum(a)

  0.006322 seconds (1 allocation: 16 bytes)


5.001462617496209e6

In [4]:
@time sum(a)

  0.007230 seconds (1 allocation: 16 bytes)


5.001462617496209e6

In [5]:
@time sum(a)

  0.008017 seconds (1 allocation: 16 bytes)


5.001462617496209e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [6]:
using Pkg
Pkg.add("BenchmarkTools")

   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `C:\Users\Tom Cummings\.julia\environments\v1.11\Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `C:\Users\Tom Cummings\.julia\environments\v1.11\Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile v1.11.0
Precompiling project...
   2621.7 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 5 seconds. 201 already precompiled.


In [7]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [8]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

Base.IOError: IOError: could not spawn `gcc -fPIC -O3 -msse3 -xc -shared -o 'C:\Users\TOMCUM~1\AppData\Local\Temp\jl_fMr6jW5yHG.dll' -`: no such file or directory (ENOENT)
Many shells expand '~' to the home directory in unquoted strings. To replicate this behavior, call `expanduser` to expand the '~' character to the user’s home directory.

In [9]:
c_sum(a)

UndefVarError: UndefVarError: `c_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [10]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

UndefVarError: UndefVarError: `c_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [11]:
c_sum(a) - sum(a)  

UndefVarError: UndefVarError: `c_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [12]:
≈  # alias for the `isapprox` function

isapprox (generic function with 13 methods)

In [13]:
?isapprox

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\Tom Cummings\OneDrive - University of Edinburgh\Documents\GitHub\Introduction-to-Julia\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X26sZmlsZQ==.jl:1:1
?isapprox
╙ ── not a unary operator

We can now benchmark the C code directly from Julia:

In [14]:
c_bench = @benchmark c_sum($a)

UndefVarError: UndefVarError: `c_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [15]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

UndefVarError: UndefVarError: `c_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

UndefVarError: UndefVarError: `c_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [17]:
using Plots
gr()

Plots.GRBackend()

In [18]:
using Statistics # bring in statistical support for standard deviations
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

UndefVarError: UndefVarError: `c_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 2. C with -ffast-math

If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.

In [19]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

Base.IOError: IOError: could not spawn `gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o 'C:\Users\TOMCUM~1\AppData\Local\Temp\jl_NQ7O7OFkLw.dll' -`: no such file or directory (ENOENT)
Many shells expand '~' to the home directory in unquoted strings. To replicate this behavior, call `expanduser` to expand the '~' character to the user’s home directory.

In [20]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

UndefVarError: UndefVarError: `c_sum_fastmath` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [21]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

UndefVarError: UndefVarError: `c_fastmath_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 3. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [22]:
using Pkg; Pkg.add("PyCall")
using PyCall

   Resolving package versions...
   Installed PyCall ─ v1.96.4
    Updating `C:\Users\Tom Cummings\.julia\environments\v1.11\Project.toml`
  [438e738f] + PyCall v1.96.4
    Updating `C:\Users\Tom Cummings\.julia\environments\v1.11\Manifest.toml`
  [438e738f] + PyCall v1.96.4
    Building PyCall → `C:\Users\Tom Cummings\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\9816a3826b0ebf49ab4926e2b18842ad8b5c8f04\build.log`


Pkg.Types.PkgError: Error building `PyCall`: 
┌ Info: Using the Python distribution in the Conda package by default.
└ To use a different Python version, set ENV["PYTHON"]="pythoncommand" and re-run Pkg.build("PyCall").
ERROR: LoadError: Conda.jl cannot be installed to its default location C:\Users\Tom Cummings\.julia\conda\3\x86_64
as Miniconda does not support the installation to a directory with a space or a
non-ASCII character on Windows. The work-around is to install Miniconda to a
user-writable directory by setting the CONDA_JL_HOME environment variable. For
example on Windows:

ENV["CONDA_JL_HOME"] = raw"C:\Conda-Julia\3"
using Pkg
Pkg.build("Conda")

The Julia session need to be restarted. More information is available at
https://github.com/JuliaPy/Conda.jl.

Stacktrace:
 [1] error(s::String)
   @ Base .\error.jl:35
 [2] _install_conda(env::String, force::Bool)
   @ Conda C:\Users\Tom Cummings\.julia\packages\Conda\zReqD\src\Conda.jl:274
 [3] _install_conda(env::String)
   @ Co

In [23]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

UndefVarError: UndefVarError: `pybuiltin` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [24]:
pysum(a)

UndefVarError: UndefVarError: `pysum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [25]:
pysum(a) ≈ sum(a)

UndefVarError: UndefVarError: `pysum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [26]:
py_list_bench = @benchmark $pysum($a)

UndefVarError: UndefVarError: `pysum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [27]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

UndefVarError: UndefVarError: `py_list_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 4. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [28]:
using Pkg; Pkg.add("Conda")
using Conda

   Resolving package versions...
    Updating `C:\Users\Tom Cummings\.julia\environments\v1.11\Project.toml`
  [8f4d0f93] + Conda v1.10.2
  No Changes to `C:\Users\Tom Cummings\.julia\environments\v1.11\Manifest.toml`
Precompiling project...
           ✗ PyCall
  0 dependencies successfully precompiled in 5 seconds. 202 already precompiled.

The following 1 direct dependency failed to precompile:

PyCall 

Failed to precompile PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0] to "C:\\Users\\Tom Cummings\\.julia\\compiled\\v1.11\\PyCall\\jl_E532.tmp".
ERROR: LoadError: PyCall not properly installed. Please run Pkg.build("PyCall")
Stacktrace:
 [1] error(s::String)
   @ Base .\error.jl:35
 [2] top-level scope
   @ C:\Users\Tom Cummings\.julia\packages\PyCall\1gn3u\src\startup.jl:44
 [3] include(mod::Module, _path::String)
   @ Base .\Base.jl:557
 [4] include(x::String)
   @ PyCall C:\Users\Tom Cummings\.julia\packages\PyCall\1gn3u\src\PyCall.jl:1
 [5] top-level scope
   @ C:\Users\Tom Cumming

In [29]:
Conda.add("numpy")

ErrorException: Conda.jl cannot be installed to its default location C:\Users\Tom Cummings\.julia\conda\3\x86_64
as Miniconda does not support the installation to a directory with a space or a
non-ASCII character on Windows. The work-around is to install Miniconda to a
user-writable directory by setting the CONDA_JL_HOME environment variable. For
example on Windows:

ENV["CONDA_JL_HOME"] = raw"C:\Conda-Julia\3"
using Pkg
Pkg.build("Conda")

The Julia session need to be restarted. More information is available at
https://github.com/JuliaPy/Conda.jl.


In [30]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

UndefVarError: UndefVarError: `pyimport` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [31]:
numpy_sum(a)

UndefVarError: UndefVarError: `numpy_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [32]:
numpy_sum(a) ≈ sum(a)

UndefVarError: UndefVarError: `numpy_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [33]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

UndefVarError: UndefVarError: `py_numpy_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 5. Python, hand-written 

In [34]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

LoadError: LoadError: UndefVarError: `@py_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at c:\Users\Tom Cummings\OneDrive - University of Edinburgh\Documents\GitHub\Introduction-to-Julia\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X65sZmlsZQ==.jl:1

In [35]:
py_hand = @benchmark $sum_py($a)

UndefVarError: UndefVarError: `sum_py` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [36]:
sum_py(a)

UndefVarError: UndefVarError: `sum_py` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [37]:
sum_py(a) ≈ sum(a)

UndefVarError: UndefVarError: `sum_py` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [38]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

UndefVarError: UndefVarError: `py_hand` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 6. Julia (built-in) 

## Written directly in Julia, not in C!

In [39]:
@which sum(a)

sum(a::AbstractArray; dims, kw...)
     @ Base reducedim.jl:982

In [40]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 575 samples with 1 evaluation per sample.
 Range (min … max):  5.607 ms … 25.103 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.120 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.642 ms ±  2.385 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▂█▄▄▄ ▂▁▄▂▂                                             
  ▃▄▅▅████████████▆█▆▅▅▅▄▄▃▃▃▁▃▃▃▁▄▂▂▃▁▃▁▂▃▂▃▂▃▂▃▂▃▁▃▁▁▂▁▁▁▂ ▃
  5.61 ms        Histogram: frequency by time        18.3 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [41]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Julia built-in" => 5.6075

# 7. Julia (hand-written) 

In [42]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [43]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 257 samples with 1 evaluation per sample.
 Range (min … max):  11.429 ms … 34.044 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     19.681 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.419 ms ±  5.465 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

        ▄█▂▄▂                   ▃▂ ▁                           
  ▃▄▆▆███████▃▃▃▆▃▁▁▄▄▄▃▅▅▆▃▆▆▇███▄██▄▆▆▃▅▄▃▃▃▃▃▃▁▁▁▃▁▁▁▁▁▃▆▃ ▃
  11.4 ms         Histogram: frequency by time        32.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [44]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Julia hand-written" => 11.4293
  "Julia built-in"     => 5.6075

# 8. Julia (hand-written w. simd) 

In [45]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [46]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 594 samples with 1 evaluation per sample.
 Range (min … max):  6.829 ms …  12.070 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.373 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.393 ms ± 903.272 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▃  ▁▂  ▅ ▂▄▁ ▂ ▃ ▁ ▂▆▃▃█▂▃▂▆▂▂  ▁   ▂ ▁                    
  ▅▆█▇▇█████████▆█████████████████████▇▅█▇█▇▇▇▇▆▄▅▄▅▆▃▃▄▃▃▁▃▃ ▅
  6.83 ms         Histogram: frequency by time        10.5 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [47]:
mysum_simd(a)

5.001462617496238e6

In [48]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "Julia hand-written simd" => 6.8288
  "Julia hand-written"      => 11.4293
  "Julia built-in"          => 5.6075

# Summary

In [49]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia built-in..............5.6
Julia hand-written simd.....6.8
Julia hand-written.........11.4
